In [157]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 16)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'

# fp_df_eval_results = 'df_eval_results'
fp_df_eval_sym_freq_results = 'df_eval_sym_freq_results'

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [158]:
# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.

###################################
# df_train, df_val, df_test = _2_split_train_val_test(df_close_clean, s_train=1, s_val=0, s_test=0)
df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
###################################

len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)
print(f'len_df_train: {len_df_train}, len_df_val: {len_df_val}, len_df_test: {len_df_test} ')

len_df_train: 1050, len_df_val: 300, len_df_test: 150 


In [159]:
col_add0 = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt']
col_add1 = ['grp(CAGR)_mean',         'grp(CAGR)_std',         'grp(CAGR)_mean/std']
col_add2 = ['grp(CAGR/UI)_mean',      'grp(CAGR/UI)_std',      'grp(CAGR/UI)_mean/std']
col_add3 = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
col_add4 = ['grp(retnStd/UI)_mean',   'grp(retnStd/UI)_std',   'grp(retnStd/UI)_mean/std']
col_add5 = ['SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI']

col_add_total = col_add0 + col_add1 + col_add2 + col_add3 + col_add4 + col_add5
print(f'col_add_total:\n{col_add_total}, total columns: {len(col_add_total)}')

col_add_total:
['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt', 'grp(CAGR)_mean', 'grp(CAGR)_std', 'grp(CAGR)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI'], total columns: 23


In [182]:
# # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results)
# df

,n_samples,days_lookbacks,days_eval,n_top_syms,syms_start,syms_end,sym_freq_cnt,grp(CAGR)_mean,grp(CAGR)_std,grp(CAGR)_mean/std,grp(CAGR/UI)_mean,grp(CAGR/UI)_std,grp(CAGR/UI)_mean/std,grp(CAGR/retnStd)_mean,grp(CAGR/retnStd)_std,grp(CAGR/retnStd)_mean/std,grp(retnStd/UI)_mean,grp(retnStd/UI)_std,grp(retnStd/UI)_mean/std,SPY_CAGR,SPY_CAGR/UI,SPY_CAGR/retnStd,SPY_retnStd/UI


In [161]:
# if store_results:  # record results to df
# my_cols = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_CAGR/UI']
# df_eval_results = pickle_load(path_data_dump, fp_df_eval_results)
df_eval_sym_freq_results = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)
# df_eval_sym_freq_results

In [162]:
verbose = True  # True prints more output
# verbose = False  # True prints more output

# write run results to df_eval_results
store_results = False
# store_results = True

# number of max lookback tuples to create for iloc start_train:end_train:end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms 
# n_samples = 400  
n_samples = 1

# for training, the number of days to lookback from iloc max-lookback end_train
# days_lookbacks = [15, 30, 60, 120]
# days_lookbacks = [30, 60, 120]
# days_lookbacks = [60, 120]
# days_lookbacks = [120]
# days_lookbacks = [60]
# days_lookbacks = [30]
# days_lookbacks = [15]
days_lookbacks = [15, 30]
# days_lookbacks = [15, 30, 60]
# days_lookbacks = [28]
# days_lookbacks = [32]
# days_lookbacks = [31]
# days_lookbacks = [29]
# days_lookbacks = [15, 29]
# days_lookbacks = [15, 31]
# days_lookbacks = [14, 30]
# days_lookbacks = [16, 32]
# days_lookbacks = [14, 28]

days_lookbacks.sort()

# number of days from end_train are used to evaluate effectiveness of the training
# days_eval = 10
# days_eval = 6
days_eval = 5
# days_eval = 4
# days_eval = 3
# days_eval = 2  


# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  #  start index of n_top_syms for evaluation
# syms_start = 1  #  start index of n_top_syms for evaluation

# syms_end = n_top_syms  #  end index of n_top_syms for evaluation
# syms_end = 1  #  end index of n_top_syms for evaluation
# syms_end = 2  #  end index of n_top_syms for evaluation
# syms_end = 3  #  end index of n_top_syms for evaluation
# syms_end = 4  #  end index of n_top_syms for evaluation
# syms_end = 5  #  end index of n_top_syms for evaluation
# syms_end = 6  #  end index of n_top_syms for evaluation
syms_end = 10  #  end index of n_top_syms for evaluation

# freq_cnt = 6  # frequency of symbol in top_set_syms_n_freq 

create a sets of iloc lookback slices (start_train:end_train:end_eval), where  
end_train - start_train = days_lookback  
end_eval - end_train = days_eval  
for example,  
if given:  
 n_samples = 2  
 days_lookbacks = [30, 60, 120]  
 days_eval = 10  
a possible result is:  
 max_lookback_slices:  
 [(150, 270, 280), (5, 125, 135)]  
 where 270-150=125-5=max(days_lookbacks), 280-270=135-125=days_eval  
 sets_lookback_slices:  
 [[(240, 270, 280), (210, 270, 280), (150, 270, 280)], [(95, 125, 135), (65, 125, 135), (5, 125, 135)]]  
  where in a set, 270-240=days_lookbacks[0], 270-210=days_lookbacks[1], 270-150=days_lookbacks[2]  
  and 270, i.e. end_train, is constant for the set  

In [163]:
# return n_samples slices
max_lookback_slices = _3_random_slices(len_df_train, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
# return n_samples * len(days_lookbacks) slices
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'max_lookback_slices:\n{max_lookback_slices}')
  print(f'sets_lookback_slices:\n{sets_lookback_slices}')

max_lookback_slices:
[(960, 990, 995)]
sets_lookback_slices:
[[(975, 990, 995), (960, 990, 995)]]


Generate lists, n_samples long, of the highest performance ranked symbols. The performance metrics are: CAGR/UI, CAGR/retnStd, retnStd/UI. n_top_syms of the best performing symbols from each metric are combined. The symbols are sorted by their number of appearances in the combined pool, and are placed in a list. A slice of the best performing symbols is selected by syms_start:syms_end, i.e. top_set_syms_n_freq[syms_start:syms_end].     

The performance metrics are calculated based on slices in sets_lookback_slices.  The first two numbers are ilocs for training. The last two numbers are ilocs for evaluation.

In [164]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    d_eval = end_eval - start_eval

    _df = df_train.iloc[start_train:end_train]
    date_start_train = _df.index[0].strftime('%Y-%m-%d')
    date_end_train = _df.index[-1].strftime('%Y-%m-%d')

    if verbose:
      print(f'days lookback:    {lookback}')
      print(f'lb_slices:        {lb_slices}')
      print(f'lb_slice:         {lb_slice}')
      print(f'days eval:        {d_eval}')    
      print(f'iloc_start_train: {start_train}')
      print(f'iloc_end_train:   {end_train}')
      print(f'date_start_train: {date_start_train}')
      print(f'date_end_train:   {date_end_train}')


    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=n_top_syms)
    grp_most_common_syms.append(most_common_syms)
    
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_common_syms: {most_common_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_common_syms: {grp_most_common_syms}')
    # grp_most_common_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
  # get symbols from top_set_syms_n_freq

###################################  
  # top_set_syms = [i[0] for i in top_set_syms_n_freq]
  top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]  
###################################  
  
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {n_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {n_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {n_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

days lookback:    15
lb_slices:        [(975, 990, 995), (960, 990, 995)]
lb_slice:         (975, 990, 995)
days eval:        5
iloc_start_train: 975
iloc_end_train:   990
date_start_train: 2021-01-26
date_end_train:   2021-02-16
perf_ranks: {'period-15': {'r_CAGR/UI': array(['MARA', 'AVXL', 'NVAX', 'CPE', 'YY', 'SM', 'BPT', 'CHRD', 'DNOW',
       'PERI', 'HUBS', 'ERF', 'KLIC', 'STNG', 'HAFC', 'OVV', 'AMKR',
       'ATKR', 'ACRS', 'ZG'], dtype=object), 'r_CAGR/retnStd': array(['MARA', 'AVXL', 'NVAX', 'CPE', 'YY', 'SM', 'BPT', 'PERI', 'MGI',
       'CHRD', 'AMKR', 'HUBS', 'ERF', 'KLIC', 'GPRE', 'OVV', 'ACRS',
       'DNOW', 'ZG', 'STNG'], dtype=object), 'r_retnStd/UI': array(['BIT', 'YY', 'HAFC', 'ZBRA', 'DNOW', 'VNDA', 'SM', 'IAC', 'SBNY',
       'MARA', 'NWSA', 'STNG', 'NWS', 'CHRD', 'IBN', 'SBLK', 'HTHT',
       'ATKR', 'CPE', 'SQ'], dtype=object)}}
most_common_syms: [('MARA', 3), ('CPE', 3), ('YY', 3), ('SM', 3), ('CHRD', 3), ('DNOW', 3), ('STNG', 3), ('AVXL', 2), ('NVAX', 2), ('BPT

In [165]:
def top_set_sym_freq_cnt(top_set_syms_n_freq):
    sym_freq_cnt_6 = []
    sym_freq_cnt_5 = []
    sym_freq_cnt_4 = []
    sym_freq_cnt_3 = []
    sym_freq_cnt_2 = []

    for sym_n_freq in top_set_syms_n_freq:
        _sym = sym_n_freq[0]
        _freq = sym_n_freq[1]
        # print(_sym, _freq)
        if _freq == 6:
            sym_freq_cnt_6.append(_sym)
        elif _freq == 5:
            sym_freq_cnt_5.append(_sym)
        elif _freq == 4:
            sym_freq_cnt_4.append(_sym)
        elif _freq == 3:
            sym_freq_cnt_3.append(_sym)          
        else:
            sym_freq_cnt_2.append(_sym)

    l_sym_freq_cnt = []
    l_sym_freq_cnt.append(sym_freq_cnt_6)
    l_sym_freq_cnt.append(sym_freq_cnt_5)
    l_sym_freq_cnt.append(sym_freq_cnt_4)
    l_sym_freq_cnt.append(sym_freq_cnt_3)    
    l_sym_freq_cnt.append(sym_freq_cnt_2)    

    # if verbose:
    #     print(f'sym_freq_cnt_6: {sym_freq_cnt_6}')
    #     print(f'sym_freq_cnt_5: {sym_freq_cnt_5}')
    #     print(f'sym_freq_cnt_4: {sym_freq_cnt_4}')
    #     print(f'sym_freq_cnt_3: {sym_freq_cnt_3}')
    #     print(f'sym_freq_cnt_2: {sym_freq_cnt_2}')

    # return sym_freq_cnt_6, sym_freq_cnt_5, sym_freq_cnt_4, sym_freq_cnt_3, sym_freq_cnt_2
    return l_sym_freq_cnt    

In [166]:
for top_set_syms_n_freq in grp_top_set_syms_n_freq:
  l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
  print(f'sym_freq_6: {l_sym_freq_cnt[0]}')
  print(f'sym_freq_5: {l_sym_freq_cnt[1]}')
  print(f'sym_freq_4: {l_sym_freq_cnt[2]}')
  print(f'sym_freq_3: {l_sym_freq_cnt[3]}')
  print(f'sym_freq_2: {l_sym_freq_cnt[4]}\n')  

sym_freq_6: ['YY']
sym_freq_5: ['ACRS', 'AVXL', 'CPE', 'MARA', 'NVAX', 'SM']
sym_freq_4: ['GPRE', 'PERI']
sym_freq_3: ['CHRD', 'DNOW', 'HAFC', 'HY', 'MGI', 'OSTK', 'SBNY', 'SSYS', 'STNG']
sym_freq_2: ['AMKR', 'ATKR']



In [167]:
from myUtils import symb_perf_stats_vectorized_v8

print('z_grp_top_set_syms_n_freq:')
z_grp_top_set_syms_n_freq = zip(max_lookback_slices, grp_top_set_syms_n_freq)

for i, (_lookback_slice, _top_set_syms_n_freq) in enumerate(z_grp_top_set_syms_n_freq):
  start_train = _lookback_slice[0]
  end_train = _lookback_slice[1]
  start_eval = end_train
  end_eval = _lookback_slice[2]

  print(f'{i + 1 } of {n_samples} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  d_start_train = df_train.index[start_train].strftime('%Y-%m-%d')
  d_end_train = df_train.index[end_train].strftime('%Y-%m-%d')
  d_start_eval = d_end_train
  d_end_eval = df_train.index[end_eval].strftime('%Y-%m-%d')
  d_df_eval_start = d_end_train
  d_df_eval_end = df_train.index[end_eval - 1].strftime('%Y-%m-%d')  
  print(f'max lookback dates: {d_start_train}, {d_end_train}, {d_end_eval}')
  print(f'df_eval dates (inclusive): {d_df_eval_start} - {d_df_eval_end}')    
  print(f'top_set_syms_n_freq: {_top_set_syms_n_freq}\n')

  l_sym_freq_cnt = top_set_sym_freq_cnt(_top_set_syms_n_freq)
  print(f'sym_freq_6: {l_sym_freq_cnt[0]}')
  print(f'sym_freq_5: {l_sym_freq_cnt[1]}')
  print(f'sym_freq_4: {l_sym_freq_cnt[2]}')
  print(f'sym_freq_3: {l_sym_freq_cnt[3]}\n')

  _sym_idx = ['SPY']
  df_SPY = df_train[start_eval:end_eval][_sym_idx]

  (
    _symbols,
    _period_yr,
    _retn,
    _DD,
    _UI,
    _MDD,
    _retnMean,
    _retnStd,
    _retnStd_div_UI,
    _CAGR,
    _CAGR_div_retnStd,
    _CAGR_div_UI,
    SPY_retnStd_d_UI,     
    SPY_CAGR,
    SPY_CAGR_d_retnStd,
    SPY_CAGR_d_UI,
  ) = symb_perf_stats_vectorized_v8(df_SPY)  

  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR: {SPY_retnStd_d_UI[0]:>17,.1f}, {SPY_CAGR_d_retnStd[0]:>17,.1f}, {SPY_CAGR_d_UI[0]:>17.1f}, {SPY_CAGR[0]:>17.1f}')

  # drop last list, with frequency count 2 or less, in l_sym_freq_cnt from zip
  zip_cnt_n_syms = zip([6,5,4,3], l_sym_freq_cnt[:-1])  
  for item in zip_cnt_n_syms:
    sym_freq_cnt = item[0]
    syms = item[1]
    if syms:  # iterate ONLY if there are symbols in syms
      df_eval = df_train[start_eval:end_eval][syms]      

      if verbose:
        print(f'start_eval: {start_eval},  date: {d_end_train}')
        print(f'end_eval:   {end_eval},  date: {d_end_eval},  df_eval last date: {d_df_eval_end}')
        print(f'frequency count of symbol(s): {sym_freq_cnt}')      
        # print(f'\ndf_eval:\n{df_eval}\n')

        syms_n_SPY = syms + ['SPY']
        df_eval_n_SPY = df_train[start_eval:end_eval][syms_n_SPY]   
        print(f'\ndf_eval_n_SPY:\n{df_eval_n_SPY}\n')

      (
        _symbols,
        _period_yr,
        _retn,
        _DD,
        _UI,
        _MDD,
        _retnMean,
        _retnStd,
        _retnStd_div_UI,
        _CAGR,
        _CAGR_div_retnStd,
        _CAGR_div_UI,
        grp_retnStd_d_UI,     
        grp_CAGR,
        grp_CAGR_d_retnStd,
        grp_CAGR_d_UI,
      ) = symb_perf_stats_vectorized_v8(df_eval)  
      print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]  :>17,.1f}, {grp_retnStd_d_UI[1]  :>17,.1f}, {grp_retnStd_d_UI[2]  :>17,.1f}')
      print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>17,.1f}, {grp_CAGR_d_retnStd[1]:>17,.1f}, {grp_CAGR_d_retnStd[2]:>17,.1f}')
      print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]     :>17,.1f}, {grp_CAGR_d_UI[1]     :>17,.1f}, {grp_CAGR_d_UI[2]     :>17,.1f}')
      print(f'grp(CAGR):         mean, std, mean/std: {grp_CAGR[0]          :>17,.1f}, {grp_CAGR[1]          :>17,.1f}, {grp_CAGR[2]          :>17,.1f}')

      if store_results:  # record results to df
        row_add0      = [n_samples, str(days_lookbacks), days_eval, n_top_syms, syms_start, syms_end, sym_freq_cnt]
        row_add1      = [grp_CAGR[0],           grp_CAGR[1],           grp_CAGR[2]]
        row_add2      = [grp_CAGR_d_UI[0],      grp_CAGR_d_UI[1],      grp_CAGR_d_UI[2]]
        row_add3      = [grp_CAGR_d_retnStd[0], grp_CAGR_d_retnStd[1], grp_CAGR_d_retnStd[2]]
        row_add4      = [grp_retnStd_d_UI[0],   grp_retnStd_d_UI[1],   grp_retnStd_d_UI[2]]
        row_add5      = [SPY_CAGR[0], SPY_CAGR_d_UI[0], SPY_CAGR_d_retnStd[0], SPY_retnStd_d_UI[0]]
        row_add_total = row_add0 + row_add1 + row_add2 + row_add3 + row_add4 + row_add5
        print(f'row_add_total: {row_add_total}')
        df_eval_sym_freq_results.loc[len(df_eval_sym_freq_results)] = row_add_total
        print(f'appended row_add to df_eval_sym_freq_results:\n{row_add_total}\n')
  
  print('='*50, '\n')  

z_grp_top_set_syms_n_freq:
1 of 1 max_lookback_slice
max_lookback_slice: (960, 990, 995)
max lookback dates: 2021-01-04, 2021-02-17, 2021-02-24
df_eval dates (inclusive): 2021-02-17 - 2021-02-23
top_set_syms_n_freq: [('YY', 6), ('ACRS', 5), ('AVXL', 5), ('CPE', 5), ('MARA', 5), ('NVAX', 5), ('SM', 5), ('GPRE', 4), ('PERI', 4), ('CHRD', 3), ('DNOW', 3), ('HAFC', 3), ('HY', 3), ('MGI', 3), ('OSTK', 3), ('SBNY', 3), ('SSYS', 3), ('STNG', 3), ('AMKR', 2), ('ATKR', 2)]

sym_freq_6: ['YY']
sym_freq_5: ['ACRS', 'AVXL', 'CPE', 'MARA', 'NVAX', 'SM']
sym_freq_4: ['GPRE', 'PERI']
sym_freq_3: ['CHRD', 'DNOW', 'HAFC', 'HY', 'MGI', 'OSTK', 'SBNY', 'SSYS', 'STNG']


SPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR:               0.4,            -166.9,             -61.4,              -0.5
start_eval: 990,  date: 2021-02-17
end_eval:   995,  date: 2021-02-24,  df_eval last date: 2021-02-23
frequency count of symbol(s): 6

df_eval_n_SPY:
                    YY         SPY
Date                              

In [168]:
df_eval_sym_freq_results.columns

Index(['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt', 'grp(CAGR)_mean', 'grp(CAGR)_std', 'grp(CAGR)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI'], dtype='object')

In [169]:
df = df_eval.copy()
df

,CHRD,DNOW,HAFC,HY,MGI,OSTK,SBNY,SSYS,STNG
Date,,,,,,,,,
2021-02-17,40.770866,10.41,16.329073,94.926125,11.43,102.510002,213.929184,48.590000,15.256752
2021-02-18,38.589703,10.08,15.811952,91.951424,10.87,97.279999,209.340439,41.290001,15.227895
2021-02-19,39.649784,9.37,16.368849,94.162872,10.87,101.199997,219.821899,43.919998,16.276434
2021-02-22,42.181755,10.49,17.045084,94.593430,8.38,94.870003,216.148895,40.290001,15.429909
2021-02-23,42.395294,10.26,17.383202,91.334953,7.26,88.139999,212.485886,35.560001,15.083601


In [170]:
df_SPY = df_SPY.copy()
df_SPY

,SPY
Date,
2021-02-17,389.833679
2021-02-18,388.174530
2021-02-19,387.489044
2021-02-22,384.508545
2021-02-23,384.975494


In [171]:
from IPython.display import display, HTML
display(HTML(df.to_html()))

,CHRD,DNOW,HAFC,HY,MGI,OSTK,SBNY,SSYS,STNG
Date,,,,,,,,,
2021-02-17,40.770866,10.41,16.329073,94.926125,11.43,102.510002,213.929184,48.590000,15.256752
2021-02-18,38.589703,10.08,15.811952,91.951424,10.87,97.279999,209.340439,41.290001,15.227895
2021-02-19,39.649784,9.37,16.368849,94.162872,10.87,101.199997,219.821899,43.919998,16.276434
2021-02-22,42.181755,10.49,17.045084,94.593430,8.38,94.870003,216.148895,40.290001,15.429909
2021-02-23,42.395294,10.26,17.383202,91.334953,7.26,88.139999,212.485886,35.560001,15.083601


In [172]:
# # Creating Empty DataFrame and Storing it in variable df
# df_eval_results = pd.DataFrame(columns=col_add_total)
pickle_dump(df_eval_sym_freq_results, path_data_dump, fp_df_eval_sym_freq_results)

In [173]:
# df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)
# df

In [174]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/

In [175]:
# # pd.crosstab(df['days_lookbacks'], df['grp(CAGR/UI)_mean'])
# tbl = df.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
#         .agg({'grp(CAGR)_mean':          ['count', 'mean', 'std',],
#               'grp(CAGR/UI)_mean':       ['mean', 'std',],
#               'grp(CAGR/retnStd)_mean':  ['mean', 'std',],
#               'SPY_CAGR':                ['mean', 'std',],
#               'SPY_CAGR/UI':             ['mean', 'std',],
#               'SPY_CAGR/retnStd':        ['mean', 'std',],                           
#               })

In [176]:
# tbl

In [177]:
# tbl['dif_CAGR_mean'] = tbl['grp(CAGR)_mean', 'mean'] - tbl['SPY_CAGR', 'mean']
# tbl['dif_CAGR/UI_mean'] = tbl['grp(CAGR/UI)_mean', 'mean'] - tbl['SPY_CAGR/UI', 'mean'] 
# tbl['dif_CAGR/retnStd_mean'] = tbl['grp(CAGR/retnStd)_mean', 'mean'] - tbl['SPY_CAGR/retnStd', 'mean'] 
# tbl['grp_CAGR/UI_mean/std'] = tbl['grp(CAGR/UI)_mean', 'mean'] / tbl['grp(CAGR/UI)_mean', 'std']  
# tbl['SPY_CAGR/UI_mean/std'] = tbl['SPY_CAGR/UI', 'mean'] / tbl['SPY_CAGR/UI', 'std'] 
# tbl['dif_CAGR/UI_mean/std'] = tbl['grp_CAGR/UI_mean/std']  - tbl['SPY_CAGR/UI_mean/std'] 
# # tbl.sort_values(by='dif_CAGR/UI_mean', ascending=False, inplace=True)
# tbl.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)
# tbl

In [178]:
if verbose:
  from IPython.display import display, HTML
  from myUtils import symb_perf_stats_vectorized_v8
  df = df_eval.copy()
  df_SPY = df_SPY.copy()  
  _my_df = [df_eval, df_SPY]
  for _df in _my_df:
    # print(f'_df:\n{_df}')
    (
      symbols,
      period_yr,
      retn,
      DD,
      UI,
      MDD,
      retnMean,
      retnStd,
      retnStd_div_UI,
      CAGR,
      CAGR_div_retnStd,
      CAGR_div_UI,
      grp_retnStd_div_UI,     
      grp_CAGR,
      grp_CAGR_div_retnStd,
      grp_CAGR_div_UI,
    ) = symb_perf_stats_vectorized_v8(_df)

    # display(HTML(_df.to_html()))
    print(f'_df:\n{_df}\n')
    print(f'symbols:\n{symbols}\n')
    print(f'period_yr:\n{period_yr}\n')
    print(f'retn:\n{retn}\n')
    print(f'DD:\n{DD}\n')
    print(f'UI:\n{UI}\n')
    print(f'MDD:\n{MDD}\n')
    print(f'retnMean:\n{retnMean}\n')
    print(f'retnStd:\n{retnStd}\n')
    print(f'retnStd_div_UI:\n{retnStd_div_UI}\n')
    print(f'CAGR:\n{CAGR}\n')
    print(f'CAGR_div_retnStd:\n{CAGR_div_retnStd}\n')
    print(f'CAGR_div_UI:\n{CAGR_div_UI}\n')
    print(f'grp_retnStd_div_UI:\n{grp_retnStd_div_UI}\n')
    print(f'grp_CAGR:\n{grp_CAGR}\n')
    print(f'grp_CAGR_div_retnStd:\n{grp_CAGR_div_retnStd}\n')
    print(f'grp_CAGR_div_UI:\n{grp_CAGR_div_UI}\n')            

_df:
                 CHRD   DNOW       HAFC         HY    MGI        OSTK        SBNY       SSYS       STNG
Date                                                                                                   
2021-02-17  40.770866  10.41  16.329073  94.926125  11.43  102.510002  213.929184  48.590000  15.256752
2021-02-18  38.589703  10.08  15.811952  91.951424  10.87   97.279999  209.340439  41.290001  15.227895
2021-02-19  39.649784   9.37  16.368849  94.162872  10.87  101.199997  219.821899  43.919998  16.276434
2021-02-22  42.181755  10.49  17.045084  94.593430   8.38   94.870003  216.148895  40.290001  15.429909
2021-02-23  42.395294  10.26  17.383202  91.334953   7.26   88.139999  212.485886  35.560001  15.083601

symbols:
Index(['CHRD', 'DNOW', 'HAFC', 'HY', 'MGI', 'OSTK', 'SBNY', 'SSYS', 'STNG'], dtype='object')

period_yr:
0.015873015873015872

retn:
                CHRD      DNOW      HAFC        HY       MGI      OSTK      SBNY      SSYS      STNG
Date                   

In [179]:
# # https://stackoverflow.com/questions/31674195/plot-normal-distribution-given-mean-and-sigma-python
# # loc is mean, scale is standard deviation
# import pylab
# import numpy as np
# from scipy.stats import norm
# # x = np.linspace(-10000,100000,1000)
# x = np.linspace(-40e+10,50e+10,1000)
# y = norm.pdf(x, loc=2.562777e+10, scale=1.036925e+11)    # loc = mean, scale = standard deviation
# # z = norm.pdf(x, loc=3.540615e+10, scale=1.194430e+11)    # for example
# # z1 = norm.pdf(x, loc=298.805901, scale=826.875749)    # for example
# # z1 = norm.pdf(x, loc=1.021825, scale=1.505096)    # for example
# pylab.plot(x,y, 'b')
# # pylab.plot(x,z, 'g')
# # pylab.plot(x,z1, 'r')
# pylab.show()

In [180]:
# # get values of _cols, where grp(CAGR/retnStd)_mean is max after filtering out inf
# _cols = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
# # _df_no_inf = df.loc[df['grp(CAGR/retnStd)_mean'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# # _idx = _df_no_inf['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# _idx = df['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# grp_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# grp_inf_replacement


In [181]:
# # get values of _cols, where SPY_CAGR/retnStd is max after filtering out inf
# _cols = ['SPY_CAGR/retnStd']
# # _df_no_inf = df.loc[df['SPY_CAGR/retnStd'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# _idx = df['SPY_CAGR/retnStd'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# SPY_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# SPY_inf_replacement